In [5]:
import numpy as np


In [1]:
with open("./released/sents.train") as f:
    texts_train = f.read()
f.close()


In [21]:
words_train = texts_train[:-1].split(sep=" ")


In [22]:
prev_POS = "[START]"

dict_train = {}
PennTreebankPOS = {"[START]": {}}
for word_train in words_train:
    tag = word_train.split("/")[-1]
    word = word_train[: -(len(tag) + 1)]

    # Multi part of speech
    if not word in dict_train.keys():
        dict_train[word] = {tag}
    else:
        if tag not in dict_train[word]:
            dict_train[word].add(tag)

    # Transition Probability Matrix
    if not tag in PennTreebankPOS.keys():  # Create new state
        PennTreebankPOS[tag] = {}
    if tag not in PennTreebankPOS[prev_POS].keys():  # a_ij from 0 to 1
        PennTreebankPOS[prev_POS][tag] = 1
    else:
        PennTreebankPOS[prev_POS][tag] += 1  # a_ij from x to x+1 (x > 0)

    prev_POS = tag


In [23]:
PennTreebankPOS

{'[START]': {'IN': 1},
 'IN': {'DT': 32393,
  '``': 553,
  'NNP': 14784,
  'IN': 2039,
  'CD': 5848,
  'JJ': 8892,
  'NN': 10739,
  'NNS': 5829,
  ',': 245,
  'VBG': 3069,
  'PRP$': 3569,
  'RB': 1400,
  'NNPS': 168,
  'JJR': 472,
  'RBS': 19,
  'PRP': 2989,
  'WP': 228,
  'TO': 216,
  ':': 25,
  'CC': 138,
  'WDT': 360,
  'WRB': 161,
  '$': 2760,
  'VBP': 27,
  'JJS': 463,
  'VBZ': 62,
  '#': 59,
  '-LRB-': 35,
  'VBN': 426,
  'FW': 20,
  'VBD': 61,
  'VB': 46,
  'PDT': 145,
  'RBR': 69,
  'EX': 157,
  'POS': 2,
  '.': 24,
  '-RRB-': 8,
  'MD': 11,
  "''": 10,
  'UH': 1,
  'WP$': 2,
  'SYM': 3,
  'LS': 1,
  'RP': 1},
 'DT': {'NNP': 9055,
  'NN': 38865,
  'JJ': 17864,
  'NNPS': 416,
  'VBN': 690,
  'NNS': 5993,
  'CD': 1878,
  'WP': 66,
  '-LRB-': 44,
  'RBR': 141,
  'VBG': 632,
  'JJR': 465,
  'RB': 853,
  'WRB': 2,
  'IN': 803,
  'RBS': 228,
  'DT': 151,
  'VBZ': 657,
  '``': 463,
  'MD': 181,
  'JJS': 762,
  'TO': 26,
  'VBP': 145,
  ',': 191,
  'PRP$': 61,
  'VBD': 180,
  '$': 757,